<a target="_blank" href="https://colab.research.google.com/github/jmanuelc87/nmp-autoavanza/blob/main/notebooks/MontePiedad_Rules.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import os
import base64
import itertools
import functools
import datetime

from pydantic import BaseModel, Field

from operator import itemgetter

from rapidfuzz import fuzz, utils

from langchain_core.tools import tool
from langchain_core.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.base import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.image import UnstructuredImageLoader

In [2]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ['OPENAI_ORG_ID'] = '***'
    os.environ['OPENAI_PROJECT_ID'] = '***'
    os.environ['OPENAI_API_KEY'] = '***'

In [3]:
llm = ChatOpenAI(
    model="*",
    # model = "gpt-4.1-mini",
    temperature=0,
    base_url='http://localhost:1234/v1',
)

In [4]:
extract = [
    {
        "role": "system",
        "content": "Eres un asistente servicial, usando OCR extraes los attributos que se te indiquen de la imagen y verificas que sean correctos comparandolo con el documento {doc_name}.",
    },
    {
        "role": "user",
        "content": [
            {"type": "image_url", "image_url": {"url": "data:image/jpg;base64,{image}"}},
            {"type": "text", "text": "El documento {doc_name}: {page_content}"},
        ],
    },
]

In [5]:
def load_image(inputs):
    """Load image from file and encode it as base64."""
    image_path = inputs["filename"]
  
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    image_base64 = encode_image(image_path)
    return image_base64

In [6]:
def load_unstructured_image(args):
    loader = UnstructuredImageLoader(args["filename"])
    data = loader.load()
    return data.pop().page_content

In [7]:
loader1 = RunnableLambda(load_unstructured_image)
loader2 = RunnableLambda(load_image)

### Extraccion Factura

In [62]:
file_url = "https://drive.google.com/file/d/1jumrzIoMQ4VJS2kCl_1E-_4bOOMlYmU6/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1jumrzIoMQ4VJS2kCl_1E-_4bOOMlYmU6"
image1_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/FAC_FRENTE/Caso 1_TK 62853-1 FAC_FRENTE_otsu.jpg"

In [63]:
prompt_extraction = ChatPromptTemplate(messages=extract)

In [64]:
class Factura(BaseModel):
    nombre_cliente: str = Field(description="El nombre del cliente")
    marca: str = Field(description="La marca")
    linea: str = Field(description="La linea")
    modelo: str = Field(description="El modelo")
    clase: str = Field(description="La clase")
    tipo: str = Field(description="El tipo")
    combustible: str = Field(description="El tipo de combustible")
    no_serie: str = Field(description="El numero de serie del motor, NIV o VIN del vehiculo")

In [65]:
llm_with_fac_structured_output = llm.with_structured_output(Factura)

In [66]:
extract_fac_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_fac_structured_output
)

In [67]:
factura1 = extract_fac_chain.invoke(input={
    "filename": image1_path,
    "doc_name": "Factura",
})

In [68]:
print(factura1)

nombre_cliente='RODRIGUEZ ELIZONDO FRANCISCO' marca='TOYOTA' linea='HILUX' modelo='2019' clase='CAMIONETA' tipo='HILUX DOB CAB SR' combustible='GASOLINA' no_serie='MROEX8DD2K0186450'


### Extraccion Tarjeta de Circulacion

In [56]:
file_url = "https://drive.google.com/file/d/1MeG8nX0r9eOkAK5xH0oN4z330WQDnYMk/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1MeG8nX0r9eOkAK5xH0oN4z330WQDnYMk"
image2_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/TC_FRENTE/Caso 1_TK 62853-4 TC_FRENTE_clean.jpg"

In [57]:
class TarjetaCirculacion(BaseModel):
    propietario: str = Field(description="El propietario o nombre del cliente")
    vehiculo: str = Field(description="El tipo de vehiculo")
    marca: str = Field(description="La marca")
    modelo: int = Field(description="El modelo del vehiculo")
    no_motor: str = Field(description="El numero de serie del motor")
    no_niv: str = Field(description="El numero de identificacion vehicular")
    expedicion: str = Field(description="La fecha de expedicion")
    vigencia: str = Field(description="La fecha de vigencia")
    placa: str = Field(description="El numero de placa")

In [58]:
llm_with_tc_structured_output = llm.with_structured_output(TarjetaCirculacion)

In [59]:
extract_tc_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_tc_structured_output
)

In [60]:
tc1 = extract_tc_chain.invoke(input={
    "filename": image2_path,
    "doc_name": "Tarjeta de Circulacion",
})

In [61]:
print(tc1)

propietario='FRANCISCO RODRIGUEZ ELIZONDO' vehiculo='HILUX 4 PUERTAS' marca='TOYOTA' modelo=2019 no_motor='2TRABOIBGS' no_niv='MROEX8DD2K0186450' expedicion='2021-09-02' vigencia='2025-09-02' placa='JW27321'


### Extraccion INE

In [44]:
file_url = "https://drive.google.com/file/d/1UKl8M0aMXJqN74iRPL9HFki8UvAdppa1/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1UKl8M0aMXJqN74iRPL9HFki8UvAdppa1"
image3_path = "./data/bronze/BASE_AUTOAVANZA/Caso 1/TK 62853-2 INE_FRENTE.png"

In [45]:
class INE(BaseModel):
    nombre: str = Field(description="El nombre de la persona")
    domicilio: str = Field(description="El domicilio de la persona")
    emision: int = Field(description="La fecha de emision")
    vigencia: int = Field(description="La fecha de vigencia")

In [46]:
llm_with_ine_structured_output = llm.with_structured_output(INE)

In [47]:
extract_ine_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_ine_structured_output
)

In [48]:
ine1 = extract_ine_chain.invoke(input={
    "filename": image3_path,
    "doc_name": "Credencial para Votar",
})

In [49]:
print(ine1)

nombre='RODRIGUEZ ELIZONDO FRANCISCO' domicilio='PRIV HIDALGO PONIENTE 546 INT 4, LOC ZAPOTILTIC CENTRO 49600 ZAPOTILTIC, JAL.' emision=2019 vigencia=2029


# Reglas para validar documentos

Se definen ciertas reglas para validar documentos como se especifica a continuacion.

1. Se debe verificar la coincidencia del nombre del solicitante en la INE, Tarjeta de circulacion y Factura.

In [ ]:
def compare_doc_names(factura: Factura, tc: TarjetaCirculacion, ine: INE) -> str:
    """Compara el nombre del cliente en los documentos Factura, Tarjeta de Circulacion y Credencial para Votar

    Args:
        factura (Factura): La factura del vehiculo
        tc (TarjetaCirculacion): La Tarjeta de circulacion
        cv (CredencialVotar): La Credencial para votar

    Returns:
        str: El resultado de la validacion es aprovado o rechazado 
    """
    wr1 = fuzz.WRatio(factura['nombre_cliente'], tc['propietario'], processor=utils.default_process)
    wr2 = fuzz.WRatio(factura['nombre_cliente'], ine['nombre'], processor=utils.default_process)
    wr3 = fuzz.WRatio(tc['propietario'], ine['nombre'], processor=utils.default_process)

    if wr1 >= 90. and wr2 >= 90. and wr3 >= 90.:
        return "Validado correctamente el nombre del cliente!"
    elif 70. <= wr1 < 90. and 70. <= wr2 < 90. and 70. <= wr3 < 90.:
        return "Favor de verificar que el nombre aparezca correctamente en los tres documentos"
    else:
        return "El nombre no aparece correctamente en los tres documentos"

2. Se debe verificar el NIV es el mismo en la tarjeta de circulación y factura

In [ ]:
def compare_doc_vin(factura: Factura, tc:TarjetaCirculacion) -> str:
    """Compara el numero de serie del vehiculo, NIV O VIN en la factura y tarjeta de circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es aprovado o rechazado
    """

    wr = fuzz.WRatio(factura['no_serie'], tc['no_niv'])

    if wr >= 90.:
        return "El NIV aparace correctamente."
    elif 75. <= wr < 90.:
        return "Favor de verificar que el NIV aparezca correctamente en los documentos"
    else:
        return "El NIV es distinto. Favor de verficar"

3. Se debe comparar los datos del vehiculo (marca, linea y modelo) en la tarjeta de circulacion y factura

In [ ]:
def compare_vehicle_data(factura: Factura, tc: TarjetaCirculacion) -> str:
    """Compara la marca, linea y modelo del vehiculo en la factura y tarjeta de circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es aprovado o rechazado
    """
    result = []

    wr = fuzz.WRatio(factura["marca"], tc["marca"])

    if wr < 90.0:
        label = f"La marca del vehiculo es distinto. Favor de verificar"
        result.append(label)

    wr = fuzz.WRatio(factura['linea'], tc['vehiculo'])

    if wr < 90.0:
        label = "La linea del vehiculo es distinta. Favor de verificar"
        result.append(label)

    wr = fuzz.WRatio(factura['modelo'], tc['modelo'])

    if wr < 90.0:
        label = "El modelo del vehiculo es distinto. Favor de verificar"
        result.append(label)

    if len(result) == 0:
        result.append("Datos del vehiculo correctos!")

    return result

In [ ]:
validate = [
    {
        "role": "system",
        "content": "Eres un asistente servicial.",
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text":"La Factura\n{factura}\n\n"},
            {"type": "text", "text":"La Tarjeta de Circulacion\n{tc}\n\n"},
            {"type": "text", "text":"La Credencial para Votar\n{cv}\n\n"},
            {"type": "text", "text":"Realiza {comparation} documentos.\n"},
        ],
    }
]

In [ ]:
validate_prompt = ChatPromptTemplate.from_messages(validate)

In [ ]:
tools=[compare_doc_names, compare_doc_vin, compare_vehicle_data]

In [ ]:
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
validations = ["la comparacion del nombre del cliente", "la comparacion del numero de serie, VIN o NIV", "la comparacion de la marca, linea y modelo del vehiculo"]

In [ ]:
def validate_documents(inputs, validations, tools):
    chain = validate_prompt | llm_with_tools | PydanticToolsParser(tools=tools)

    for validation in validations:
        result = chain.invoke(input={
            "comparation": validation,
            **inputs,
        })

        if isinstance(result[0], list):
            result = list(itertools.chain.from_iterable(result))
        
        yield result

In [ ]:
validation = RunnableLambda(functools.partial(validate_documents, validations=validations, tools=tools))

In [ ]:
chain = {
    "factura": {
        "filename": itemgetter("factura"),
        "doc_name": lambda x: "Factura",
    }
    | extract_fac_chain,
    "tc": {
        "filename": itemgetter("tarjeta_circulacion"),
        "doc_name": lambda x: "Tarjeta de Circulacion",
    }
    | extract_tc_chain,
    "cv": {
        "filename": itemgetter("credencial_votar"),
        "doc_name": lambda x: "Credencial para Votar"
    }
    | extract_ine_chain,
} | validation

In [ ]:
for e in chain.stream({
    "factura": image1_path,
    "tarjeta_circulacion": image2_path,
    "credencial_votar": image3_path,
}):
    print(e)